# 闭包函数

## 使用参数传参 

In [4]:
def func(x):
    print(x)

func(1)
func(1)
func(1)

1
1
1


## 包给函数

In [6]:
def outter(x):
    
    #定义inner函数输出x
    def inner():
        print(x)
    return inner

f = outter(1)
f()
f()
f()
print(f'查看闭包元素：{f} ')

1
1
1
查看闭包元素：<function outter.<locals>.inner at 0x000002B8374D25E8> 


## 应用--爬虫

In [7]:
import requests


def get(url):
    response = requests.get(url)
    print(f"done: {url}")
    
get('https://www.baidu.com')
get('https://www.baidu.com')
get('https://www.baidu.com')


get('https://www.cnblogs.com/linhaifeng')
get('https://www.cnblogs.com/linhaifeng')
get('https://www.cnblogs.com/linhaifeng')

done: https://www.baidu.com
done: https://www.baidu.com
done: https://www.baidu.com
done: https://www.cnblogs.com/linhaifeng
done: https://www.cnblogs.com/linhaifeng
done: https://www.cnblogs.com/linhaifeng


In [8]:
#对上述代码进行改进  简介明了 简化传参
import requests


def outter(url):
    def get():
        response = requests.get(url)
        print(f"done: {url}")
    return get

baidu=outter('https://www.baidu.com')
python = outter('https://www.python.org')

baidu()
baidu()

python()
python()

done: https://www.baidu.com
done: https://www.baidu.com
done: https://www.python.org
done: https://www.python.org


# 装饰器 
+ 装饰器的实现必须遵循两大原则：
    + 不修改被装饰对象的源代码
    + 不修改被装饰对象的调用方式

## 无参装饰器 

In [9]:
#我们希望在源代码中加入时间模块
import time


def index():
    start = time.time()
    print('welcome to index')
    time.sleep(1)
    end = time.time()
    print(F"index run time is {start-end}")


index()


welcome to index
index run time is -1.0018134117126465


In [12]:
#如果对每一个每个函数进行操作 那么会非常复杂
import time


def index():
    print('welcome to index')
    time.sleep(1)


def f2():
    print('welcome to index')
    time.sleep(1)


start = time.time()
index()
end = time.time()
print(F"index run time is {start-end}")

start = time.time()
f2()
end = time.time()
print(F"f2 run time is {start-end}")
"""我们需要在每次执行函数时进行计时"""

welcome to index
index run time is -1.0027251243591309
welcome to index
f2 run time is -1.0008063316345215


'我们需要在每次执行函数时进行计时'

### 改变调用方式

In [13]:
import time


def index():
    print('welcome to index')
    time.sleep(1)


def time_count(func):
    start = time.time()
    func()
    end = time.time()
    print(f"{func} time is {start-end}")


time_count(index)


welcome to index
<function index at 0x000002B83732D9D8> time is -1.0018465518951416


### 第二种传参方式：包给函数-外包

In [14]:
import time


def index():
    print('welcome to index')
    time.sleep(1)

#包给函数
def time_count(func):
    # func = 最原始的index
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f"{func} time is {start-end}")
    return wrapper

# f = time_count(index)
# f()


index = time_count(index)  # index为被装饰函数的内存地址，即index = wrapper
index()  # wrapper()


welcome to index
<function index at 0x000002B8374D2C18> time is -1.0000450611114502


### 完善装饰器
上述的装饰器，最后调用index()的时候，其实是在调用wrapper()，因此如果原始的index()有返回值的时候，wrapper()函数的返回值应该和index()的返回值相同，也就是说，我们需要同步原始的index()和wrapper()方法的返回值。

In [16]:
import time


def index():
    print('welcome to index')
    time.sleep(1)

    return 123


def time_count(func):
    # func = 最原始的index
    def wrapper():
        start = time.time()
        res = func()
        end = time.time()
        print(f"{func} time is {start-end}")

        return res
    return wrapper

#res跟踪index()输出
index = time_count(index)
res = index()
print(f"res: {res}")


welcome to index
<function index at 0x000002B8374D2288> time is -1.0015192031860352
res: 123


**如果原始的index()方法需要传参，那么我们之前的装饰器是无法实现该功能的，由于有wrapper()=index()，所以给wrapper()方法传参即可。**

In [20]:
import time


def index():
    print('welcome to index')
    time.sleep(1)

    return 123


def home(name):
    print(f"welcome {name} to home page")
    time.sleep(1)

    return name


def time_count(func):
    # func = 最原始的index
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print(f"{func} time is {start-end}")

        return res
    return wrapper


home = time_count(home)

res = home('egon')
print(f"res: {res}")


welcome 13 to home page
<function home at 0x000002B83729ED38> time is -1.0014503002166748
res: 13


### 装饰器语法糖
在被装饰函数正上方，并且是单独一行写上**@装饰器名** 我们就不需要另外函数进行说明

In [21]:
import time


def time_count(func):
    # func = 最原始的index
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print(f"{func} time is {start-end}")

        return res
    return wrapper


@time_count  # home = time_count(home)
def home(name):
    print(f"welcome {name} to home page")
    time.sleep(1)

    return name


@time_count  # index = time_count(index)
def index():
    print('welcome to index')
    time.sleep(1)

    return 123


res = home('egon')
print(f"res: {res}")


welcome egon to home page
<function home at 0x000002B837EB2CA8> time is -1.0007948875427246
res: egon


### **装饰器模板**

In [ ]:
def deco(func):
    def wrapper(*args,**kwargs):
        res = func(*args,**kwargs)
        return res
    return wrapper

## 有参装饰器
[有参装饰器]([https://www.cnblogs.com/nickchen121/p/10771174.html])

# 迭代器
+ 可迭代的对象：Python内置str、list、tuple、dict、set、file都是可迭代对象。
+ 特点：
    + 内置有__iter__方法的都叫可迭代的对象。
+ 迭代器对象：可迭代的对象执行__iter__方法得到的返回值。并且可迭代对象会有一个__next__方法
    + 内置\_\_next\_\_方法，执行该方法会拿到迭代器对象中的一个值
    + 内置有\_\_iter\_\_方法，执行该方法会拿到迭代器本身
    + 文件本身就是迭代器对象。

## 可迭代对象

In [23]:
# x = 1.__iter__  # SyntaxError: invalid syntax

# 以下都是可迭代的对象

name = 'nick'.__iter__
lis = [1, 2].__iter__
tup = (1, 2).__iter__
dic = {'name': 'nick'}.__iter__
s1 = {'a', 'b'}.__iter__

## 迭代器对象

In [24]:
# 不依赖索引的数据类型迭代取值
dic = {'a': 1, 'b': 2, 'c': 3}
iter_dic = dic.__iter__()
print(iter_dic.__next__())
print(iter_dic.__next__())
print(iter_dic.__next__())
# print(iter_dic.__next__())  # StopIteration:

a
b
c


In [25]:
#逐个取值比较繁杂 我们使用循环
s = 'hello'
iter_s = s.__iter__()

while True:
    try:
        print(iter_s.__next__())
    except StopIteration:
        break


h
e
l
l
o


## for循环原理
+ for循环称为迭代器循环，in后必须是可迭代的对象。